<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# LinkedIn - Send interactions from post URL to HubSpot notes

**Tags:** #linkedin #hubspot #interactions #post #url #send #notes

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-08-16 (Created: 2023-08-16)

**Description:** This notebook automates the process of sending people interactions (like or comment) on a LinkedIn post URL to a contact notes in HubSpot. If the people does not exist in HubSpot, she/he will be created.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
import naas
from naas_drivers import linkedin, hubspot
import pandas as pd

### Setup variables
**Mandatory**

[Learn how to get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `linkedin_url`: This variable represents the LinkedIn profile URL.

[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.

**Optional**
- `properties`: List of properties (hubspot internal names) you want to get from contact. By default, you will get: email, firstname, lastname, createdate, lastmodifieddate, hs_object_id. To list of contact properties, you can use this template: "HubSpot/HubSpot_List_contact_properties.ipynb" stored in https://github.com/jupyter-naas/awesome-notebooks.

In [ ]:
# Mandatory
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
linkedin_url = "https://www.linkedin.com/feed/update/urn:li:activity:7100209398274220032"  # EXAMPLE "https://www.linkedin.com/in/XXXXXX/"
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"

# Optional
properties = []

## Model

### Get post likes

In [ ]:
df_likes = linkedin.connect(li_at, JSESSIONID).post.get_likes(linkedin_url)
print("Number of likes: ", len(df_likes))
df_likes.head(5)

### Get post comments

In [ ]:
df_comments = linkedin.connect(li_at, JSESSIONID).post.get_comments(linkedin_url)
print("Number of comments: ", len(df_comments))
df_comments.head(5)

### Create database of LinkedIn profiles

In [ ]:
def create_profiles_db(
    df_likes,
    df_comments
):
    # Init
    df = pd.DataFrame()
    
    # Concat db
    to_keep = [
        "PROFILE_ID",
        "PROFILE_URL",
        "PUBLIC_ID",
        "FIRSTNAME",
        "LASTNAME",
        "FULLNAME",
        "OCCUPATION",
        "PROFILE_PICTURE",
    ]
    df = pd.concat([df_likes, df_comments])[to_keep].drop_duplicates(to_keep)
    return df.reset_index(drop=True)

df_profiles = create_profiles_db(
    df_likes,
    df_comments
)
print("LinkedIn profiles:", len(df_profiles))
df_profiles.head(5)

### Get all contacts

In [ ]:
df_contacts = hubspot.connect(hs_access_token).contacts.get_all(properties)
print("Rows:", len(df_contacts))
df_contacts.head(1)

## Output

### Display result

In [ ]:
# Print interactions
print(interactions)